In [15]:
import bs4
import base64
import re
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import os

In [16]:
def tokenize(s):
    #print(s)
    s = ' '.join(re.split('https?://.*\\W', s))
    return list(filter(lambda x: len(x) > 1, re.split('\\W', s)))

In [17]:
def text_from_doc(doc):
    a = []
    b = []
    #print(type(doc))

    if isinstance(doc, bs4.element.Tag):
        #print(doc.name)
        if doc.name in ['text', 'textarea', 'p']:
            for child in doc.contents:
                if not isinstance(child, bs4.element.Tag):
                    a += tokenize(child)
                else:
                    kek = text_from_doc(child)
                    a += kek[0]
                    b += kek[1]
        elif doc.name == 'a' or doc.name == 'link':
            if doc.has_attr('href'):
                b.append(doc['href'])
            for child in doc.contents:
                if not isinstance(child, bs4.element.Tag):
                    a += tokenize(child)
                else:
                    kek = text_from_doc(child)
                    a += kek[0]
                    b += kek[1]
        else:
            for child in doc.contents:
                kek = text_from_doc(child)
                a += kek[0]
                b += kek[1]
    return a, b

In [18]:
files_in_dir = !(ls ../byweb_for_course/ | grep byweb)

In [ ]:
bytelen_f = open('bytelens.out', 'w')
wordlen_f = open('wordlens.out', 'w')
doc_f = open('docs.out', 'w')

freqs = dict()
idfs = dict()

for file_name in files_in_dir:
    inf = open(os.path.join('../byweb_for_course/', file_name), 'r') 
    l = inf.readlines()
    for i in range(1, len(l) - 1, 3):#len(l) - 1, 3):
        s = l[i] + l[i + 1] + l[i + 2]
        soup = bs4.BeautifulSoup(s, "lxml")
        docid = soup.docid.contents[0]
        url = base64.b64decode(soup.docurl.contents[0]).decode("cp1251")
        content = base64.b64decode(soup.content.contents[0]).decode("cp1251")
        soup1 = bs4.BeautifulSoup(content, 'lxml')
        
        a = []
        b = []
        for doc in soup1:
            kek = text_from_doc(doc)
            a += kek[0]
            b += kek[1]
        
        jn = ' '.join(a)
        lemmas = !cat a | /home/mikhail/mystem
        lemmas = lemmas[0]
        tokens = re.split('{|}', lemmas)
        tokens1 = []
        for i in range(1, len(tokens), 2):
            tokens1.append(re.split('\|', tokens[i])[0])
        
        idf = dict()
        
        for k in idf.keys():
            idfs[k] += idf[k]
        
        for t in tokens1:
            idf[t] = 1
            if t not in freqs:
                freqs[t] = 1
            else:
                freqs[t] = freqs[t] + 1
        
        
        doc_f.write('{}\t{}\t{}\t{}\n'.format(docid, url, a, b))
        wordlen_f.write('{}\n'.format(len(a)))
        bytelen_f.write('{}\n'.format(len(soup1.get_text())))
        
doc_f.close()
wordlen_f.close()
bytelen_f.close()

In [121]:
np.average(bytelens)

199280.0

In [122]:
np.average(wordlens)

70.92592592592592

In [123]:
kek = dict()
for ln in wordlens:
    key = (ln // 100) * 100
    if key not in kek:
        kek[key] = 1
    else:
        kek[key] = kek[key] + 1

In [124]:
df = pd.DataFrame()
df["keys"] = kek.keys()
df["values"] = kek.values()

px.bar(df, x="keys", y="values")

In [106]:
a = ' '.join(docs[7][2])
b = !echo $a | /home/mikhail/mystem

In [91]:
docs[7][1]

'http://brest.linux.by/forum/index.php?action=vthread&forum=8&topic=126'

In [56]:
s = 'abacaba http://krk.lol/arbidol caa'
re.split('https?://.*\\W', s)

['abacaba ', 'caa']

In [92]:
re.split('({|})', b[0])

['Начало']

In [93]:
b[0]

'Начало'

In [94]:
re.split('(^|}).*?{', b[0])

['Начало']

In [107]:
b

['Начало{начало|начинать}Ответить{отвечать}Статистика{статистика|статистик}Регистрация{регистрация}Поиск{поиск}FAQ{FAQ??}Язык{язык}Аноним{аноним}thxburjuinrwz{thxburjuinrwz??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}Аноним{аноним}mziwj{mziwj??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}html{html??}Аноним{аноним}mziwj{mziwj??}']

In [117]:
kek = re.split('{|}', b[0])
kek1 = ''
for i in range(1, len(kek), 2):
    kek1 += re.split('\|', kek[i])[0] + ' '

In [118]:
kek1

'начало отвечать статистика регистрация поиск FAQ?? язык аноним thxburjuinrwz?? html?? html?? html?? html?? html?? html?? html?? аноним mziwj?? html?? html?? html?? html?? html?? html?? html?? html?? аноним mziwj?? '